In [4]:
from ultralytics import YOLO


model = YOLO("./YOLO/best_YOLOV11.pt")

print(model.names)

{0: 'Elipse', 1: 'Cloud', 2: 'Other', 3: 'Rectangle', 4: 'Sea_Urchin', 5: 'Thorn'}


In [5]:
from ultralytics import YOLO

model = YOLO("./YOLO/yolov11_best_5_class.pt")
print(model.names)

{0: 'Elipse', 1: 'Cloud', 2: 'Rectangle', 3: 'Other', 4: 'Thorn'}


In [ ]:
import json

# Load the JSON data from the uploaded file
file_name = "./file_testing/2 copy.json"
with open(file_name, 'r') as f:
    data = json.load(f)

extracted_bounding_boxes = []

# Extract from "bubble" objects
if "bubble" in data:
    for i, bubble in enumerate(data["bubble"]):
        if all(k in bubble for k in ["x1", "y1", "x2", "y2"]):
            extracted_bounding_boxes.append({
                "type": "bubble",
                "id": i,
                "x1": bubble["x1"],
                "y1": bubble["y1"],
                "x2": bubble["x2"],
                "y2": bubble["y2"]
            })
        else:
            print(f"Warning: Bubble object at index {i} is missing coordinate keys.")

# Extract from "text" objects (using their main x1, y1, x2, y2, not the detailed 'mask' polygon)
if "text" in data:
    for i, text_item in enumerate(data["text"]):
        if all(k in text_item for k in ["x1", "y1", "x2", "y2"]):
            extracted_bounding_boxes.append({
                "type": "text",
                "id": i,
                "x1": text_item["x1"],
                "y1": text_item["y1"],
                "x2": text_item["x2"],
                "y2": text_item["y2"]
            })
        else:
            print(f"Warning: Text object at index {i} is missing coordinate keys.")

if "onomatope" in data:
    for i, text_item in enumerate(data["onomatope"]):
        if all(k in text_item for k in ["x1", "y1", "x2", "y2"]):
            extracted_bounding_boxes.append({
                "type": "onomatope",
                "id": i,
                "x1": text_item["x1"],
                "y1": text_item["y1"],
                "x2": text_item["x2"],
                "y2": text_item["y2"]
            })
        else:
            print(f"Warning: Text object at index {i} is missing coordinate keys.")


# Print the extracted bounding boxes
if extracted_bounding_boxes:
    print("Extracted Bounding Boxes for Masking:")
    for box in extracted_bounding_boxes:
        print(box)
    
    # Save to a new JSON file
    output_filename = "extracted_bounding_boxes copy.json"
    with open(output_filename, 'w') as outfile:
        json.dump(extracted_bounding_boxes, outfile, indent=2)
    print(f"\nSuccessfully extracted {len(extracted_bounding_boxes)} bounding boxes.")
    print(f"The extracted bounding boxes have been saved to '{output_filename}'")
else:
    print("No bounding boxes found or data format is not as expected.")

# For demonstration, if you wanted to use these to create a simple mask on a blank image
# (requires image dimensions and a library like Pillow or OpenCV), here's a conceptual idea:

# from PIL import Image, ImageDraw
#
# # Assuming an image size (e.g., from the context of where these coordinates came from)
# # This is a placeholder, you'd need actual image dimensions
# image_width = 2000
# image_height = 2000
#
# # Create a new blank image (e.g., black background)
# mask_image = Image.new('L', (image_width, image_height), 0) # 'L' for grayscale, 0 for black
# draw = ImageDraw.Draw(mask_image)
#
# for box in extracted_bounding_boxes:
#     # Draw a white rectangle for each bounding box
#     # The coordinates are (x1, y1, x2, y2)
#     draw.rectangle([box['x1'], box['y1'], box['x2'], box['y2']], fill=255) # 255 for white
#
# # Save or display the mask
# mask_image_filename = "generated_mask.png"
# try:
#    mask_image.save(mask_image_filename)
#    print(f"A sample mask image has been saved to '{mask_image_filename}'")
#    print("Note: This mask is based on placeholder image dimensions and shows all bounding boxes.")
# except Exception as e:
#    print(f"Could not save sample mask image: {e}")
#    print("This might be due to the bounding box coordinates exceeding the placeholder image dimensions.")
#    print("Actual image dimensions are needed for a correct visual mask.")

# Create a sample mask for the first bounding box if coordinates are within reasonable limits
# for a demo image.
from PIL import Image, ImageDraw

# Determine max coordinates to define a somewhat reasonable placeholder image size
max_x = 0
max_y = 0
if extracted_bounding_boxes:
    for box in extracted_bounding_boxes:
        max_x = max(max_x, box.get('x1', 0), box.get('x2', 0))
        max_y = max(max_y, box.get('y1', 0), box.get('y2', 0))

# Add some padding
image_width = max_x + 100 if max_x > 0 else 1500 # Default if no boxes or zero coords
image_height = max_y + 100 if max_y > 0 else 2000 # Default

if extracted_bounding_boxes:
    mask_image = Image.new('L', (image_width, image_height), 0)
    draw = ImageDraw.Draw(mask_image)
    
    for box in extracted_bounding_boxes:
         # Ensure coordinates are within the image bounds before drawing
        x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']
        
        # Clamp coordinates to be within image dimensions
        # This prevents errors if coordinates are outside the determined image_width/height
        clamped_x1 = max(0, min(x1, image_width -1))
        clamped_y1 = max(0, min(y1, image_height -1))
        clamped_x2 = max(0, min(x2, image_width -1))
        clamped_y2 = max(0, min(y2, image_height -1))

        if clamped_x1 < clamped_x2 and clamped_y1 < clamped_y2: # Ensure valid rectangle
             draw.rectangle([clamped_x1, clamped_y1, clamped_x2, clamped_y2], fill=255)
        else:
            print(f"Skipping box due to invalid/clamped dimensions: {box}")


    mask_image_filename = "generated_mask copy.png"
    try:
        mask_image.save(mask_image_filename)
        print(f"A sample mask image showing all bounding boxes has been saved to '{mask_image_filename}'")
        print(f"The mask image dimensions are: width={image_width}, height={image_height}")
    except Exception as e:
        print(f"Could not save the combined sample mask image: {e}")
else:
    print("No bounding boxes were extracted, so no mask image generated.")

Extracted Bounding Boxes for Masking:
{'type': 'bubble', 'id': 0, 'x1': 978, 'y1': 1551, 'x2': 1264, 'y2': 1971}
{'type': 'bubble', 'id': 1, 'x1': 768, 'y1': 442, 'x2': 977, 'y2': 775}
{'type': 'bubble', 'id': 2, 'x1': 1103, 'y1': 976, 'x2': 1299, 'y2': 1281}
{'type': 'bubble', 'id': 3, 'x1': 551, 'y1': 1518, 'x2': 876, 'y2': 1950}
{'type': 'bubble', 'id': 4, 'x1': 1281, 'y1': 938, 'x2': 1432, 'y2': 1241}
{'type': 'bubble', 'id': 5, 'x1': 872, 'y1': 1096, 'x2': 1051, 'y2': 1396}
{'type': 'bubble', 'id': 6, 'x1': 759, 'y1': 184, 'x2': 1018, 'y2': 524}
{'type': 'bubble', 'id': 7, 'x1': 593, 'y1': 83, 'x2': 717, 'y2': 309}
{'type': 'bubble', 'id': 8, 'x1': 670, 'y1': 590, 'x2': 825, 'y2': 853}
{'type': 'bubble', 'id': 9, 'x1': 230, 'y1': 1581, 'x2': 421, 'y2': 1896}
{'type': 'bubble', 'id': 10, 'x1': 1223, 'y1': 1515, 'x2': 1394, 'y2': 1833}
{'type': 'bubble', 'id': 11, 'x1': 563, 'y1': 910, 'x2': 797, 'y2': 1362}
{'type': 'bubble', 'id': 12, 'x1': 427, 'y1': 1060, 'x2': 641, 'y2': 1483}


In [ ]:

# Extract from "bubble" objects
if "bubble" in data:
    for i, bubble in enumerate(data["bubble"]):
        if all(k in bubble for k in ["x1", "y1", "x2", "y2"]):
            extracted_bounding_boxes.append({
                "type": "bubble",
                "id": i,
                "x1": bubble["x1"],
                "y1": bubble["y1"],
                "x2": bubble["x2"],
                "y2": bubble["y2"]
            })
        else:
            print(f"Warning: Bubble object at index {i} is missing coordinate keys.")

# Extract from "text" objects (using their main x1, y1, x2, y2, not the detailed 'mask' polygon)
if "text" in data:
    for i, text_item in enumerate(data["text"]):
        if all(k in text_item for k in ["x1", "y1", "x2", "y2"]):
            extracted_bounding_boxes.append({
                "type": "text",
                "id": i,
                "x1": text_item["x1"],
                "y1": text_item["y1"],
                "x2": text_item["x2"],
                "y2": text_item["y2"]
            })
        else:
            print(f"Warning: Text object at index {i} is missing coordinate keys.")

if "onomatope" in data:
    for i, text_item in enumerate(data["onomatope"]):
        if all(k in text_item for k in ["x1", "y1", "x2", "y2"]):
            extracted_bounding_boxes.append({
                "type": "onomatope",
                "id": i,
                "x1": text_item["x1"],
                "y1": text_item["y1"],
                "x2": text_item["x2"],
                "y2": text_item["y2"]
            })
        else:
            print(f"Warning: Text object at index {i} is missing coordinate keys.")


# Print the extracted bounding boxes
if extracted_bounding_boxes:
    print("Extracted Bounding Boxes for Masking:")
    for box in extracted_bounding_boxes:
        print(box)
    
    # Save to a new JSON file
    output_filename = "extracted_bounding_boxes copy.json"
    with open(output_filename, 'w') as outfile:
        json.dump(extracted_bounding_boxes, outfile, indent=2)
    print(f"\nSuccessfully extracted {len(extracted_bounding_boxes)} bounding boxes.")
    print(f"The extracted bounding boxes have been saved to '{output_filename}'")
else:
    print("No bounding boxes found or data format is not as expected.")

# For demonstration, if you wanted to use these to create a simple mask on a blank image
# (requires image dimensions and a library like Pillow or OpenCV), here's a conceptual idea:

# from PIL import Image, ImageDraw
#
# # Assuming an image size (e.g., from the context of where these coordinates came from)
# # This is a placeholder, you'd need actual image dimensions
# image_width = 2000
# image_height = 2000
#
# # Create a new blank image (e.g., black background)
# mask_image = Image.new('L', (image_width, image_height), 0) # 'L' for grayscale, 0 for black
# draw = ImageDraw.Draw(mask_image)
#
# for box in extracted_bounding_boxes:
#     # Draw a white rectangle for each bounding box
#     # The coordinates are (x1, y1, x2, y2)
#     draw.rectangle([box['x1'], box['y1'], box['x2'], box['y2']], fill=255) # 255 for white
#
# # Save or display the mask
# mask_image_filename = "generated_mask.png"
# try:
#    mask_image.save(mask_image_filename)
#    print(f"A sample mask image has been saved to '{mask_image_filename}'")
#    print("Note: This mask is based on placeholder image dimensions and shows all bounding boxes.")
# except Exception as e:
#    print(f"Could not save sample mask image: {e}")
#    print("This might be due to the bounding box coordinates exceeding the placeholder image dimensions.")
#    print("Actual image dimensions are needed for a correct visual mask.")

# Create a sample mask for the first bounding box if coordinates are within reasonable limits
# for a demo image.
from PIL import Image, ImageDraw

# Determine max coordinates to define a somewhat reasonable placeholder image size
max_x = 0
max_y = 0
if extracted_bounding_boxes:
    for box in extracted_bounding_boxes:
        max_x = max(max_x, box.get('x1', 0), box.get('x2', 0))
        max_y = max(max_y, box.get('y1', 0), box.get('y2', 0))

# Add some padding
image_width = max_x + 100 if max_x > 0 else 1500 # Default if no boxes or zero coords
image_height = max_y + 100 if max_y > 0 else 2000 # Default

if extracted_bounding_boxes:
    mask_image = Image.new('L', (image_width, image_height), 0)
    draw = ImageDraw.Draw(mask_image)
    
    for box in extracted_bounding_boxes:
         # Ensure coordinates are within the image bounds before drawing
        x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']
        
        # Clamp coordinates to be within image dimensions
        # This prevents errors if coordinates are outside the determined image_width/height
        clamped_x1 = max(0, min(x1, image_width -1))
        clamped_y1 = max(0, min(y1, image_height -1))
        clamped_x2 = max(0, min(x2, image_width -1))
        clamped_y2 = max(0, min(y2, image_height -1))

        if clamped_x1 < clamped_x2 and clamped_y1 < clamped_y2: # Ensure valid rectangle
             draw.rectangle([clamped_x1, clamped_y1, clamped_x2, clamped_y2], fill=255)
        else:
            print(f"Skipping box due to invalid/clamped dimensions: {box}")


    mask_image_filename = "generated_mask copy.png"
    try:
        mask_image.save(mask_image_filename)
        print(f"A sample mask image showing all bounding boxes has been saved to '{mask_image_filename}'")
        print(f"The mask image dimensions are: width={image_width}, height={image_height}")
    except Exception as e:
        print(f"Could not save the combined sample mask image: {e}")
else:
    print("No bounding boxes were extracted, so no mask image generated.")

In [16]:
import json
import cv2
import numpy as np
import os

def draw_polygon_mask(image, points, color=(255, 255, 255)):
    if points:
        pts = np.array(points, np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(image, [pts], color)

def process_json_and_mask(image_path, json_path, output_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Failed to load image: {image_path}")
        return
    image =image * 0

    with open(json_path, 'r') as f:
        data = json.load(f)

    # Process only masks
    for section in ["text", "onomatope"]:
        for item in data.get(section, []):
            mask = item.get("mask")
            if mask:
                draw_polygon_mask(image, mask)

    cv2.imwrite(output_path, image)
    print(f"✅ Masked image saved to {output_path}")

def process_all_json_in_folder(json_folder, image_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(json_folder):
        if filename.endswith(".json"):
            json_path = os.path.join(json_folder, filename)
            base_name = os.path.splitext(filename)[0]

            # Look for image with any common extension
            for ext in [".jpg", ".jpeg", ".png", ".webp"]:
                image_path = os.path.join(image_folder, base_name + ext)
                if os.path.exists(image_path):
                    break
            else:
                print(f"⚠️ Image not found for {filename}, skipping.")
                continue

            output_path = os.path.join(output_folder, base_name + "_masked.jpg")
            process_json_and_mask(image_path, json_path, output_path)

# Example usage:
# process_all_json_in_folder("json_folder", "images", "output")
json_folder = "../uploads/6ed0b1ad/edited"
image_folder = "../uploads/6ed0b1ad/image"
output_path = "../uploads/6ed0b1ad/mask"

process_all_json_in_folder(json_folder, image_folder, output_path)



✅ Masked image saved to ../uploads/6ed0b1ad/mask\2_masked.jpg


In [ ]:
import json
with open("translation_config.json", 'r') as f:
    translation_config = json.load(f)

a = translation_config.get("font_choices")

for i in range(0,5):
    print(a[str(i)])

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 64746: character maps to <undefined>

In [19]:
with open("3_translated.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

translate = data['translations']

for translation in translate:
    coords = translation["coords"]
    x1, y1, x2, y2 = coords["x1"], coords["y1"], coords["x2"], coords["y2"]
    english_text = translation["english_text"]
    inside_bubble = translation["inside_bubble"]
    cls = translation.get("bubble_class",6) if not None else 6  # Default to 6 if not specified
    temp_face = None
    font_path = None
    print(cls)
    # print(type(cls))

0
4
0
None
0
0
None
0
None
0
3
3
